In [1]:
import scipy, matplotlib
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.optimize import minimize

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time
import warnings

In [3]:
training = pd.read_csv('training.csv')

In [4]:
test = pd.read_csv('test.csv')

In [5]:
def sse(par, data):
    x = data['Time']
    z = data['Pressure']
    y = data['Speed']
    w = data['Part_size_original']
    b = par[0] + par[5]*z
    c = par[1] + par[6]*z
    d = par[2] + par[7]*z
    e = par[3] + par[8]*z
    f = par[4] + par[9]*z
    if np.any(e < 1e-10):
        return float('inf')
    
    y_pred = c + ((d - c + f*x)/(1+np.exp(b*(np.log(x)-np.log(e)))))
    s_se = np.sum((y - y_pred)**2)
    return(s_se)

def extended_bc5(test_, coef):
    x = test_['Time']
    z = test_['Pressure']
    w = test_['Part_size_original']
    b = coef[0] + coef[5]*z
    c = coef[1] + coef[6]*z
    d = coef[2] + coef[7]*z
    e = coef[3] + coef[8]*z
    f = coef[4] + coef[9]*z
    y_ = c + ((d - c + f*x)/(1+np.exp(b*(np.log(x)-np.log(e)))))
    return(y_)


In [6]:
def sse_nl(par, data):
    x = data['Time']
    z = data['Pressure']
    y = data['Speed']
    w = data['Part_size_original']
    b = par[0] + par[5]*z + par[10]*(z**2)
    c = par[1] + par[6]*z + par[11]*(z**2)
    d = par[2] + par[7]*z + par[12]*(z**2)
    e = par[3] + par[8]*z + par[13]*(z**2)
    f = par[4] + par[9]*z + par[14]*(z**2)
    y_pred = c + ((d - c + f*x)/(1+np.exp(b*(np.log(x)-np.log(e)))))
    s_se = np.sum((y - y_pred)**2)
    return(s_se)

def extended_bc5_nl(test_, coef):
    x = test_['Time']
    z = test_['Pressure']
    w = test_['Part_size_original']
    b = coef[0] + coef[5]*z + coef[10]*(z**2)
    c = coef[1] + coef[6]*z + coef[11]*(z**2)
    d = coef[2] + coef[7]*z + coef[12]*(z**2)
    e = coef[3] + coef[8]*z + coef[13]*(z**2)
    f = coef[4] + coef[9]*z + coef[14]*(z**2)
    y_ = c + ((d - c + f*x)/(1+np.exp(b*(np.log(x)-np.log(e)))))
    return(y_)

In [ ]:
## CO2
# predictors: pressure, time
print("---Building model using extended BC.5 for CO2---")
# init_coeff = [6.4790, 17.8232, 170.9843, 19.9785, -9.9553, 1, 1, 1, 1, 1]
init_coeff = [6.4790, 17.8232, 170.9843, 19.9785, -9.9553, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

start_time = time.time()
warnings.simplefilter("ignore", RuntimeWarning)
coeff_opt = minimize(sse_nl, init_coeff, args = (training[training["Gas"]=="CO2"],), method='BFGS', options = {'maxiter' : 10000})
end_time = time.time()
print(f"Time taken by function: {end_time - start_time} seconds")

y_pred = extended_bc5_nl(test[test["Gas"]=="CO2"], coeff_opt.x)

print("---Results---")
rmse_val = np.sqrt(mean_squared_error(test[test["Gas"]=="CO2"]["Speed"], y_pred))
mae_val = mean_absolute_error(test[test["Gas"]=="CO2"]["Speed"], y_pred)
print(f"Custom BC.5 model, RMSE: {rmse_val}")
print(f"And its Mean Absolute error: {mae_val}")

---Building model using extended BC.5 for CO2---


In [36]:
coeff_opt

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 254738796.28506905
        x: [ 2.187e+00  7.822e+00  5.986e+02 -1.751e+00 -2.223e+02
             2.147e+00  2.231e+01 -8.426e+01  3.377e+00  8.145e+01]
      nit: 71
      jac: [-2.000e+00  0.000e+00  2.000e+00  2.000e+00 -2.000e+00
             2.000e+00 -2.000e+00  0.000e+00 -2.000e+00 -2.000e+00]
 hess_inv: [[ 9.650e-08 -4.645e-08 ... -2.234e-09 -9.912e-10]
            [-4.645e-08  2.264e-07 ...  2.316e-08 -5.116e-08]
            ...
            [-2.234e-09  2.316e-08 ...  7.962e-09 -3.205e-09]
            [-9.912e-10 -5.116e-08 ... -3.205e-09  2.134e-08]]
     nfev: 1023
     njev: 93

In [7]:
## Helium
# predictors: pressure, time
print("---Building model using extended BC.5 for Helium---")
# init_coeff = [3.8302, 41.9722, 415.3618, 3.1784, -67.4586, 1, 1, 1, 1, 1]
init_coeff = [3.8302, 41.9722, 415.3618, 3.1784, -67.4586, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

start_time = time.time()
# to ignore RuntimeWarning
warnings.simplefilter("ignore", RuntimeWarning)
coeff_opt = minimize(sse_nl, init_coeff, args = (training[training["Gas"]=="Helium"],), method='BFGS', options = {'maxiter' : 10000, 'disp': True})
end_time = time.time()
print(f"Time taken by function: {end_time - start_time} seconds")

y_pred = extended_bc5_nl(test[test["Gas"]=="Helium"], coeff_opt.x)

print("---Results---")
rmse_val = np.sqrt(mean_squared_error(test[test["Gas"]=="Helium"]["Speed"], y_pred))
mae_val = mean_absolute_error(test[test["Gas"]=="Helium"]["Speed"], y_pred)
print(f"Custom BC.5 model, RMSE: {rmse_val}")
print(f"And its Mean Absolute error: {mae_val}")

---Building model using extended BC.5 for Helium---


C:\Users\GP-Preds\anaconda3\envs\mytf\lib\site-packages\scipy\optimize\_minimize.py:708: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


         Current function value: 119294448.053477
         Iterations: 76
         Function evaluations: 1824
         Gradient evaluations: 114
Time taken by function: 37.712088108062744 seconds
---Results---
Custom BC.5 model, RMSE: 52.19701121916192
And its Mean Absolute error: 30.380414191236195


In [37]:
coeff_opt

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 477249541.85902596
        x: [ 3.767e+00  4.213e+01  4.156e+02  8.366e-01 -6.690e+01
             1.641e+00  1.476e+00  1.600e+00  1.460e+00  2.735e+00]
      nit: 5
      jac: [-1.862e+06 -1.816e+06 -2.173e+06 -8.570e+07 -5.978e+06
            -3.630e+06 -4.738e+06 -4.636e+06 -2.054e+08 -1.351e+07]
     nfev: 88
     njev: 8
 hess_inv: <10x10 LbfgsInvHessProduct with dtype=float64>

In [15]:
## Luft
# predictors: pressure, time
print("---Building model using extended BC.5 for Air (Luft)---")
# init_coeff = [2.5525, 27.7128, 249.7527, 18.0972, -16.2367, 1, 1, 1, 1, 1]
init_coeff = [2.5525, 27.7128, 249.7527, 18.0972, -16.2367, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

start_time = time.time()
# to ignore RuntimeWarning
warnings.simplefilter("ignore", RuntimeWarning)
coeff_opt = minimize(sse_nl, init_coeff, args = (training[training["Gas"]=="Luft"],), method='BFGS', options = {'maxiter' : 10000, 'disp': True})
end_time = time.time()
print(f"Time taken by function: {end_time - start_time} seconds")

y_pred = extended_bc5_nl(test[test["Gas"]=="Luft"], coeff_opt.x)

print("---Results---")
rmse_val = np.sqrt(mean_squared_error(test[test["Gas"]=="Luft"]["Speed"], y_pred))
mae_val = mean_absolute_error(test[test["Gas"]=="Luft"]["Speed"], y_pred)
print(f"Custom BC.5 model, RMSE: {rmse_val}")
print(f"And its Mean Absolute error: {mae_val}")

---Building model using extended BC.5 for Air (Luft)---


C:\Users\GP-Preds\anaconda3\envs\mytf\lib\site-packages\scipy\optimize\_minimize.py:708: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


         Current function value: 192618397.775039
         Iterations: 131
         Function evaluations: 2211
         Gradient evaluations: 201
Time taken by function: 226.20241332054138 seconds
---Results---
Custom BC.5 model, RMSE: 14.26767821484053
And its Mean Absolute error: 7.632580708526298


In [75]:
print(coeff_opt.x)

[ -6.89019738 -94.30699633   0.2528591  -17.3376954   -5.04151834
   6.63083808  58.25613681 132.47975004  22.65269336  -1.88838878]


In [8]:
print(coeff_opt.x)

[-1.10255204e+01 -6.38718062e+02  3.71625670e+01 -7.83675746e+00
  2.48092893e+03  1.49761220e+01  6.45545080e+02  3.66924541e+02
  9.56973741e+00 -2.56317576e+03 -3.58633854e+00 -1.47597919e+02
 -8.05283556e+01 -1.88168253e+00  6.38053110e+02]
